#### Exécuter si vous n'avez pas déjà installer ces deux packages

In [ ]:
#!pip install gensim
#!pip install python-Levenshtein

#### Import des librairies

In [8]:
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#### Import des données

In [2]:
df = pd.read_json('quotes-2020.json.bz2', compression='bz2', nrows=100000, lines=True)

In [4]:
df.head(1)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E


In [131]:
df[df['speaker']=='None'].head(1)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E


In [ ]:
good_quotes = []
tokens = "climate warming environment".split()
new_vector = model.infer_vector(tokens)

def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        for row in chunk.iterrows():
            if 'limate change' in str(row[1]['quotation']) :
                good_quotes.append(row[1]['quotation'])
          
        
with pd.read_json('quotes-2020.json.bz2', lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        process_chunk(chunk)

In [169]:
good_quotes[:5]

["And it's also in the context of decades of utter failure on climate change. And now we're staring down a very dark future.",
 'Many make the link today between their experience of violent conflicts and climate change,',
 'the National Energy and Climate Plans are how we chart the course towards climate neutrality. We have no time to lose in our fight against climate change so we really need to respect our own deadlines for action.',
 'This budget also allows us to earmark £ 500,000 to ensure funding is available for the recommendations that will come from our environment and climate change taskforce, helping us to achieve our ambition of being carbon neutral by 2030 and ensuring our finances are robust enough to meet the local and global challenges of the future.',
 'As a state, we will pursue every option available under existing law to combat the effects of climate change and put Oregon on a path we can be proud to leave behind for our children.']

In [1]:
good_quotes[:3]

NameError: name 'good_quotes' is not defined

#### 1. Doc2Vec

#### On va créer une liste de TaggedDocument. Chaque TaggedDocument contient un Document, et un Tag (identifiant unique).

Deux manières de créer les Tagged-Data, on voit que la deuxieme est mieux car elle permet de mieux nettoyer les quotations : 

In [7]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['quotation'])]

In [28]:
tagged_data[0]

TaggedDocument(words=['[', 'd', ']', 'espite', 'the', 'efforts', 'of', 'the', 'partners', 'to', 'create', 'a', 'non-political', 'award', 'that', 'unites', 'the', 'jewish', 'people', ',', 'some', 'have', 'incorrectly', 'interpreted', 'the', 'participation', 'of', 'the', 'office', 'of', 'the', 'prime', 'minister', 'in', 'the', 'genesis', 'prize', 'as', 'bringing', 'a', 'political', 'dimension', 'to', 'this', 'important', 'initiative', ','], tags=['0'])

In [52]:
vocab_quotes = df.quotation.apply(gensim.utils.simple_preprocess)
tagged_data_2 = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(vocab_quotes)]

In [53]:
tagged_data_2[0]

TaggedDocument(words=['espite', 'the', 'efforts', 'of', 'the', 'partners', 'to', 'create', 'non', 'political', 'award', 'that', 'unites', 'the', 'jewish', 'people', 'some', 'have', 'incorrectly', 'interpreted', 'the', 'participation', 'of', 'the', 'office', 'of', 'the', 'prime', 'minister', 'in', 'the', 'genesis', 'prize', 'as', 'bringing', 'political', 'dimension', 'to', 'this', 'important', 'initiative'], tags=[0])

In [67]:
# On définit le modèle, il y a beaucoup de paramètres à étudier pour optimiser
#model = gensim.models.Doc2Vec(vector_size=300, window=10, min_count=5, workers=4, epochs=20)
model = gensim.models.Doc2Vec(vector_size=100, dbow_words= 1, dm=0,  window=5, seed=1337, min_count=5, workers=4,alpha=0.025, min_alpha=0.025)

In [68]:
# On construit le vocab nécessaire au training 
model.build_vocab(tagged_data_2)

In [130]:
# On entraine le modèle
model.train(tagged_data_2, total_examples=model.corpus_count, epochs=10)

In [175]:
# On cherche les 10 documents les plus similaires à celui-ci. 

tokens = "climate warming environment".split()
new_vector = model.infer_vector(tokens)
sims = model.dv.most_similar([new_vector], topn=30)

In [178]:
model.docvecs.similarity('climate warming environment', 'Many make the link today between their experience of violent conflicts and climate change')

<ipython-input-178-699a2bb19f84>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs.similarity('climate warming environment', 'Many make the link today between their experience of violent conflicts and climate change')


KeyError: "Key 'climate warming environment' not present"

In [177]:
sims

[(23771, 0.7676342725753784),
 (47475, 0.7420128583908081),
 (6518, 0.7162148356437683),
 (43734, 0.7111215591430664),
 (85019, 0.7109125256538391),
 (95234, 0.7011255621910095),
 (86231, 0.7006449699401855),
 (65002, 0.6982077360153198),
 (21455, 0.6878529191017151),
 (40763, 0.6850039958953857),
 (68263, 0.6796166300773621),
 (5304, 0.676025390625),
 (2302, 0.6682032346725464),
 (74714, 0.6679670810699463),
 (19419, 0.6671484708786011),
 (64469, 0.6649736762046814),
 (74302, 0.6552955508232117),
 (10474, 0.6490005254745483),
 (65040, 0.648915708065033),
 (98878, 0.6482448577880859),
 (65260, 0.6475870013237),
 (58040, 0.6474145650863647),
 (25068, 0.6432352662086487),
 (99586, 0.6427844762802124),
 (29448, 0.642483115196228),
 (79853, 0.6411502361297607),
 (87628, 0.6401574611663818),
 (35887, 0.6374573707580566),
 (17582, 0.636782705783844),
 (52509, 0.6339145302772522)]

In [176]:
for elt in sims :
    print(str(df.iloc[[elt[0]]]['quotation']))

23771    in response to the climate changing,
Name: quotation, dtype: object
47475    Responses to Climate Change,
Name: quotation, dtype: object
6518    Risky Business: Climate and the Macroeconomy,
Name: quotation, dtype: object
43734    getting serious about the climate emergency,
Name: quotation, dtype: object
85019    Climate World War III.
Name: quotation, dtype: object
95234    clear climate change fuelled the ferocity of t...
Name: quotation, dtype: object
86231    sensitive to the current climate.
Name: quotation, dtype: object
65002    Investors are increasingly... recognising that...
Name: quotation, dtype: object
21455    life ‐ sustaining climate system.
Name: quotation, dtype: object
40763    Adapting to climate change isn't enough.
Name: quotation, dtype: object
68263    The drivers in the community are for decarboni...
Name: quotation, dtype: object
5304    Canada's Global Role in the Climate Emergency.
Name: quotation, dtype: object
2302    Weather is a serious issue!


In [117]:
print(str(df.iloc[[10474 ]]['quotation']))

10474    epically outmatched by a global pandemic.
Name: quotation, dtype: object


In [ ]:
# Bon ben apparement ca marche assez bien, comment faire mieux désormais ? 

#### 2. Word2Vec

In [16]:
vocab_quotes = df.quotation.apply(gensim.utils.simple_preprocess)

In [29]:
vocab_quotes[0]

['espite',
 'the',
 'efforts',
 'of',
 'the',
 'partners',
 'to',
 'create',
 'non',
 'political',
 'award',
 'that',
 'unites',
 'the',
 'jewish',
 'people',
 'some',
 'have',
 'incorrectly',
 'interpreted',
 'the',
 'participation',
 'of',
 'the',
 'office',
 'of',
 'the',
 'prime',
 'minister',
 'in',
 'the',
 'genesis',
 'prize',
 'as',
 'bringing',
 'political',
 'dimension',
 'to',
 'this',
 'important',
 'initiative']

In [20]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [21]:
model.build_vocab(vocab_quotes, progress_per=1000)

In [22]:
model.train(vocab_quotes, total_examples=model.corpus_count, epochs=model.epochs)

(8831377, 11833700)

In [124]:
model.wv.most_similar("climate", topn=50)

[('deforestation', 0.578827440738678),
 ('inequality', 0.530994713306427),
 ('change', 0.5236321687698364),
 ('global', 0.5184160470962524),
 ('declines', 0.5175962448120117),
 ('disparities', 0.5151465535163879),
 ('structural', 0.5150886178016663),
 ('robustly', 0.5074840784072876),
 ('unforgiving', 0.50663161277771),
 ('wildfire', 0.5015254616737366),
 ('mitigating', 0.4986288547515869),
 ('racial', 0.49859723448753357),
 ('environmental', 0.4983930289745331),
 ('equality', 0.49219849705696106),
 ('destructive', 0.48925095796585083),
 ('confederation', 0.4890865683555603),
 ('radical', 0.48764461278915405),
 ('regulation', 0.4872642457485199),
 ('initiatives', 0.48353496193885803),
 ('unruly', 0.48303091526031494),
 ('catalysts', 0.480881005525589),
 ('combating', 0.48057907819747925),
 ('jeopardizes', 0.47917208075523376),
 ('impending', 0.4784373342990875),
 ('nuclear', 0.47744154930114746),
 ('extremes', 0.4771021902561188),
 ('silencing', 0.4768330156803131),
 ('ecological', 0.4

In [ ]:
# Ca marche aussi, mais y a des mots qui ne sont pas reliés QUE au changement climatique comme 'policy' et qui
# ne peuvent pas servir de clé pour déterminer à eux tout seul qu'une quote est sur le changement climatique.

### Conclusion :  Se concentrer sur la première approche Doc2Vec à mon avis, en mettant plus de données dans l'algorithme, chercher à trouver les paramètres optimaux de train, améliorer le pré-processing en dégageant les mots tronqués, et en réajustant continuellement pour avoir la meilleure accuracy possible. 

#### Trouver comment importer toute la donnée en même temps ? Ou une maniere d'utiliser toute la donnée pour entrainer le modèle en tout cas. 

#### Aussi pourquoi quand on print kes